In [1]:
######################
## import libraries ##
######################
import pandas as pd
import numpy as np    
np.random.seed(10815657)
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
#########################################
## generate simulated dataset function ##
#########################################
def simulate_dataset(n=100000):
    A_split = 0.4
    C1_split = 0.2
    C2_split = 0.65

    df_A1 = pd.DataFrame()
    df_A1['A'] = np.ones(int(n*A_split))
    df_A1['C1'] = np.random.uniform(0, 1, int(n*A_split))
    df_A1['C2'] = np.random.uniform(0, 1, int(n*A_split))
    df_A1.loc[df_A1['C1']<C1_split, 'C1'] = 0
    df_A1.loc[df_A1['C1']>=C1_split, 'C1'] = 1
    df_A1.loc[df_A1['C2']<C2_split, 'C2'] = 0
    df_A1.loc[df_A1['C2']>=C2_split, 'C2'] = 1

    df_A0 = pd.DataFrame()
    df_A0['A'] = np.zeros(int(n*(1-A_split)))
    df_A0['C1'] = np.random.uniform(0, 1, int(n*(1-A_split)))
    df_A0['C2'] = np.random.uniform(0, 1, int(n*(1-A_split)))
    df_A0.loc[df_A0['C1']<(1-C1_split), 'C1'] = 0
    df_A0.loc[df_A0['C1']>=(1-C1_split), 'C1'] = 1
    df_A0.loc[df_A0['C2']<(1-C2_split), 'C2'] = 0
    df_A0.loc[df_A0['C2']>=(1-C2_split), 'C2'] = 1

    df = pd.concat([df_A0, df_A1], axis=0).reset_index(drop=True)
    df['error'] = np.random.normal(0, 0.25, df.shape[0]) 
    del df_A1, df_A0

    B0 = -0.23
    B1 = 1.23
    B2 = 1.56
    B3 = 0.87
    df['Y'] = B0 + (B1*df['A']) + (B2*df['C1']) + (B3*df['C2']) + df['error']
    
    print("The true outcome model is:")
    print('E[Y|A,C1,C2] = ' + str(B0) + ' + ' + str(B1) + '*A + ' + str(B2) + '*C1 + ' + str(B3) + '*C2')
    
    return(df)

In [3]:
## get simulated dataset
df_original = simulate_dataset(n=100000)

The true outcome model is:
E[Y|A,C1,C2] = -0.23 + 1.23*A + 1.56*C1 + 0.87*C2


In [4]:
###############################################
## fit and return standardized outcome model ##
###############################################
def fit_standardized_outcome_model(df, model_string, fit_bounds=True, print_result=True):
    df_original = df.copy()
    df_A0 = df_original.copy()
    df_A1 = df_original.copy()
    df_A0['A'] = 0
    df_A1['A'] = 1
    outcome_model = smf.ols(formula=model_string, data=df).fit()
    df_A0['Y_fit'] = outcome_model.predict(df_A0)
    df_A1['Y_fit'] = outcome_model.predict(df_A1)
    mean_unexposed = df_A0['Y_fit'].mean()
    mean_exposed = df_A1['Y_fit'].mean()
    effect_difference = mean_exposed - mean_unexposed
        
    if(fit_bounds):
        effect_difference_list = []
        for i in range(0, 100):
            df_sub = df_original.sample(n=df_original.shape[0], replace=True)
            df_A0_sub = df_sub.copy()
            df_A1_sub = df_sub.copy()
            df_A0_sub['A'] = 0
            df_A1_sub['A'] = 1
            OM_sub = smf.ols(formula=model_string, data=df_sub).fit()
            df_A0_sub['Y_fit'] = OM_sub.predict(df_A0_sub)
            df_A1_sub['Y_fit'] = OM_sub.predict(df_A1_sub)
            mean_unexposed_sub = df_A0_sub['Y_fit'].mean()
            mean_exposed_sub = df_A1_sub['Y_fit'].mean()
            effect_difference_sub = mean_exposed_sub - mean_unexposed_sub
            effect_difference_list.append(effect_difference_sub)
            del df_sub, df_A0_sub, df_A1_sub, OM_sub, mean_unexposed_sub, mean_exposed_sub, effect_difference_sub
   
        variance = np.var(effect_difference_list)
        standard_error = np.sqrt(variance)
        bound_size = 1.96*standard_error
        lower_bound = effect_difference - bound_size
        upper_bound = effect_difference + bound_size
    
    if(print_result):
        print(outcome_model.summary())
        print("The standardized mean in those with A=0 is: " + str(mean_unexposed))
        print("The standardized mean in those with A=1 is: " + str(mean_exposed))
        if(fit_bounds):
            print("The standardized effect difference is " + str(effect_difference) + " with 95% CI (" + str(lower_bound) + ' , ' + str(upper_bound) + ')')
        else:
            print("The standardized effect difference is " + str(effect_difference))
    
    return(df, outcome_model)

In [5]:
## fit and return correctly specified outcome model
df_OM_CS, OM_CS = fit_standardized_outcome_model(df_original.copy(), 'Y ~ A + C1 + C2')

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.958
Model:                            OLS   Adj. R-squared:                  0.958
Method:                 Least Squares   F-statistic:                 7.620e+05
Date:                Thu, 03 Dec 2020   Prob (F-statistic):               0.00
Time:                        11:33:19   Log-Likelihood:                -3353.7
No. Observations:              100000   AIC:                             6715.
Df Residuals:                   99996   BIC:                             6753.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2272      0.002   -147.829      0.0

In [6]:
## fit and return incorrectly specified outcome model
df_OM_IS, OM_IS = fit_standardized_outcome_model(df_original.copy(), 'Y ~ A + C2')

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                 1.161e+05
Date:                Thu, 03 Dec 2020   Prob (F-statistic):               0.00
Time:                        11:34:46   Log-Likelihood:            -1.0193e+05
No. Observations:              100000   AIC:                         2.039e+05
Df Residuals:                   99997   BIC:                         2.039e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0806      0.004     20.230      0.0

In [7]:
##############################################
## fit and return Marginal Strucutral Model ##
##############################################
def fit_MSM_model(df, model_string, print_result=True):
    PS_model = smf.glm(model_string, data=df, family=sm.families.Binomial()).fit()
    df['Propensity_Score'] = PS_model.predict(df)
    Marg_Prob_Intervention = df.loc[df['A']==1, :].reset_index(drop=True).shape[0] / df.shape[0]
    df['Unstabalized_Weights'] = None
    df.loc[df['A']==1, 'Unstabalized_Weights'] = 1 / (df.loc[df['A']==1, 'Propensity_Score'])
    df.loc[df['A']==0, 'Unstabalized_Weights'] = 1 / (1 - df.loc[df['A']==0, 'Propensity_Score'])
    df['Stabalized_Weights'] = None
    df.loc[df['A']==1, 'Stabalized_Weights'] = (Marg_Prob_Intervention) / (df.loc[df['A']==1, 'Propensity_Score'])
    df.loc[df['A']==0, 'Stabalized_Weights'] = (1 - Marg_Prob_Intervention) / (1 - df.loc[df['A']==0, 'Propensity_Score'])
    MSM = smf.wls('Y ~ A', data=df, weights=np.array(df['Stabalized_Weights'], dtype=np.float64)).fit()
    
    mean_unexposed = MSM.params[0]
    mean_exposed = MSM.params[0] + MSM.params[1]
    effect_difference = mean_exposed - mean_unexposed
    lower_bound = MSM.conf_int()[0][1]
    upper_bound = MSM.conf_int()[1][1]

    if(print_result):
        print(MSM.summary())
        print("The MSM mean in those with A=0 is: " + str(mean_unexposed))
        print("The MSM mean in those with A=1 is: " + str(mean_exposed))
        print("The MSM effect difference is " + str(effect_difference) + " with 95% CI (" + str(lower_bound) + ' , ' + str(upper_bound) + ')')
    return(df, MSM)

In [8]:
## fit and return correctly specified Marginal Structural Model
df_MSM_CS, MSM_CS = fit_MSM_model(df_original.copy(), 'A ~ C1 + C2')

                            WLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.331
Model:                            WLS   Adj. R-squared:                  0.331
Method:                 Least Squares   F-statistic:                 4.949e+04
Date:                Thu, 03 Dec 2020   Prob (F-statistic):               0.00
Time:                        11:34:56   Log-Likelihood:            -1.3710e+05
No. Observations:              100000   AIC:                         2.742e+05
Df Residuals:                   99998   BIC:                         2.742e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9175      0.003    262.333      0.0

In [9]:
## fit and return incorrectly specified Marginal Structural Model
df_MSM_IS, MSM_IS = fit_MSM_model(df_original.copy(), 'A ~ C2')

                            WLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.638
Model:                            WLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                 1.764e+05
Date:                Thu, 03 Dec 2020   Prob (F-statistic):               0.00
Time:                        11:35:02   Log-Likelihood:            -1.2161e+05
No. Observations:              100000   AIC:                         2.432e+05
Df Residuals:                   99998   BIC:                         2.432e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5411      0.003    165.952      0.0

In [10]:
############################################
## fit and return Doubly Robust Estimator ##
############################################
def fit_Doubly_Robust_Estimator(df, df_MSM, outcome_model, specification_type, fit_bounds=False):
    df_A0 = df.copy()
    df_A0['A'] = 0
    df_A0['Y_fit'] = outcome_model.predict(df_A0)
    df_A1 = df.copy()
    df_A1['A'] = 1
    df_A1['Y_fit'] = outcome_model.predict(df_A1)
    
    exposed = (1/df.shape[0])*((df['A']*((df_A1['Y']-df_A1['Y_fit']) / (df_MSM['Propensity_Score']))) + df_A1['Y_fit']).sum()
    unexposed = (1/df.shape[0])*(((1-df['A'])*((df_A0['Y']-df_A0['Y_fit']) / (1-df_MSM['Propensity_Score']))) + df_A0['Y_fit']).sum()
    
    DR_Causal_Effect = exposed - unexposed
    
    if(fit_bounds):
        lower_bound, upper_bound = estimate_DRE_CI(df.copy(), DR_Causal_Effect, specification_type)
        print('The Doubly Robust Estimate for the Causal Effect of A on Y is: ' + str(DR_Causal_Effect) + " with 95% CI (" + str(lower_bound) + ' , ' + str(upper_bound) + ')')
    return(DR_Causal_Effect)

##############################################################
## get standard error estimates for Doubly Robust Estimator ##
##############################################################
def estimate_DRE_CI(df_original, point_estimate, specification_type):
    DRE_list=[]
    
    for i in range(0,100):
        df = df_original.sample(n=df_original.shape[0], replace=True)
        
        if(specification_type=='OM & MSM correctly specified'):
            df_OM, OM = fit_standardized_outcome_model(df.copy(), 'Y ~ A + C1 + C2', fit_bounds=False, print_result=False)
            df_MSM, MSM = fit_MSM_model(df.copy(), 'A ~ C1 + C2', print_result=False)
        if(specification_type=='OM incorrectly specified, MSM correctly specified'):
            df_OM, OM = fit_standardized_outcome_model(df.copy(), 'Y ~ A + C2', fit_bounds=False, print_result=False)
            df_MSM, MSM = fit_MSM_model(df.copy(), 'A ~ C1 + C2', print_result=False)
        if(specification_type=='OM correctly specified, MSM incorrectly specified'):
            df_OM, OM = fit_standardized_outcome_model(df.copy(), 'Y ~ A + C1 + C2', fit_bounds=False, print_result=False)
            df_MSM, MSM = fit_MSM_model(df.copy(), 'A ~ C2', print_result=False)
        if(specification_type=='OM & MSM incorrectly specified'):
            df_OM, OM = fit_standardized_outcome_model(df.copy(), 'Y ~ A + C2', fit_bounds=False, print_result=False)
            df_MSM, MSM = fit_MSM_model(df.copy(), 'A ~ C2', print_result=False)
        
        DRE = fit_Doubly_Robust_Estimator(df.copy(), df_MSM, OM, specification_type, fit_bounds=False)
        DRE_list.append(DRE)
        
        del df, df_OM, OM, df_MSM, MSM, DRE
    
    variance = np.var(DRE_list)
    standard_error = np.sqrt(variance)
    bound_size = 1.96*standard_error
    lower_bound = point_estimate - bound_size
    upper_bound = point_estimate + bound_size
    
    #print('The 95% CI is: (' + str(lower_bound) + ' , ' + str(upper_bound) + ')')
        
    return(lower_bound, upper_bound)  

In [11]:
## Doubly Robust Estimator, with both the Standardized Outcome Model & Marginal Structural Model correctly specified
DRE_CS_point_estimate = fit_Doubly_Robust_Estimator(df_original.copy(), df_MSM_CS, OM_CS, specification_type='OM & MSM correctly specified', fit_bounds=True)

The Doubly Robust Estimate for the Causal Effect of A on Y is: 1.2273660077867443 with 95% CI (1.2227483264361219 , 1.2319836891373668)


In [12]:
## Doubly Robust Estimator, with the Standardized Outcome Model misspecified
DRE_OM_IS_point_estimate = fit_Doubly_Robust_Estimator(df_original.copy(), df_MSM_CS, OM_IS, specification_type='OM incorrectly specified, MSM correctly specified', fit_bounds=True)

The Doubly Robust Estimate for the Causal Effect of A on Y is: 1.2236769956032711 with 95% CI (1.2181937402171832 , 1.229160250989359)


In [13]:
## Doubly Robust Estimator, with the Marginal Structural Model misspecified
DRE_MSM_IS_point_estimate = fit_Doubly_Robust_Estimator(df_original.copy(), df_MSM_IS, OM_CS, specification_type='OM correctly specified, MSM incorrectly specified', fit_bounds=True)

The Doubly Robust Estimate for the Causal Effect of A on Y is: 1.2278114671931366 with 95% CI (1.2237678339601503 , 1.231855100426123)


In [14]:
## Doubly Robust Estimator, with both the Standardized Outcome Model & Marginal Structural Model misspecified
DRE_IS_point_estimate = fit_Doubly_Robust_Estimator(df_original.copy(), df_MSM_IS, OM_IS, specification_type='OM & MSM incorrectly specified', fit_bounds=True)

The Doubly Robust Estimate for the Causal Effect of A on Y is: 2.1655698807162245 with 95% CI (2.157191073007725 , 2.173948688424724)
